In [ ]:
# This page is for provisioning an ESP32 GPS device for the wingtip or base-station.  
# For the main unit logger with the other sensors, look at HangDataLogger


In [3]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [4]:
%ls

Listing directory '/'.
      139    boot.py


In [5]:
%sendtofile --source UDPblackviewphone.py
%sendtofile --source ubxminimal.py

Sent 66 lines (2004 bytes) to ubxminimal.py.


In [6]:
%sendtofile deviceletter.txt
A

Sent 1 lines (1 bytes) to deviceletter.txt.


In [15]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [16]:
%sendtofile hotspots.txt

BV6000         beckaaaa    192.168.43.1   9042
ES_3041        43900000    192.168.43.1   9042
JulianS9       beckaaaa    192.168.43.1   9042
DoESLiverpool  decafbad00  node-red.dynamicdevices.co.uk   4006

Sent 4 lines (204 bytes) to hotspots.txt.


In [42]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [8]:
%sendtofile main.py

import time, machine

# delay long enough for Ctrl-C before the wlan bricks it for debugging
# reset_cause=1:PWRON_RESET, 2:HARD, 5:SOFT (brownout detected)
pled = machine.Pin(2, machine.Pin.OUT)
for i in range(2*(machine.reset_cause())):
    pled.value(1-(i%2))
    time.sleep_ms(400)  

# Set up flashing LED timer (better control than PWM)
pgled = machine.Pin(23, machine.Pin.OUT)
timeracc = 0
timermax = 1000
timerlight = 100
timeradd = 50
def timercallback(t):
    global timeracc
    timeracc = (timeracc + timeradd)%timermax
    pgled.value(int(timeracc<timerlight))
timer = machine.Timer(-1)
timer.init(period=50, mode=machine.Timer.PERIODIC, callback=timercallback)

# Serial connection to the UBlox GPS device
from ubxminimal import uartUBX, initUBX
initUBX()

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(200)

print("RESET_CAUSE", machine.reset_cause())
deviceletter = open("deviceletter.txt", "r").read()
print("deviceletter", deviceletter)

from UDPblackviewphone import connectActivePhone
while True:
    androidipnumber, portnumber = connectActivePhone(pled)
    if androidipnumber is not None:
        break
    for j in range(3):
        for i in range(7):
            pled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

for i in range(21):
    pled.value(i%2)
    time.sleep_ms(80)

import socket, uselect

ubxbuffer = bytearray(1000)
mubxbuffer = memoryview(ubxbuffer)
timelastledsignal = 0
totalbytes = 0
obj = None
while True:
    timermax = 4000  # slow down signal to show it's broken
    try:
        ss = socket.socket()
        ss.settimeout(1)
        print(ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        poller = uselect.poll()
        poller.register(uartUBX, uselect.POLLIN)
        poller.register(s, uselect.POLLIN)
        while True:
            for obj, evt in poller.ipoll(100):
                if obj == uartUBX and evt == uselect.POLLIN:
                    n = uartUBX.readinto(ubxbuffer)
                    if n is not None:
                        s.write(mubxbuffer[:n])
                        pled.value(1-pled.value())
                        totalbytes += n
                    if time.ticks_ms() - timelastledsignal > 2000:
                        timermax = 3000  # slow down signal to show it's broken

                if obj == s and evt == uselect.POLLIN:
                    timelastledsignal = time.ticks_ms()
                    timermax = 1000
                    l = s.readline()
                    try:
                        print(l)
                        timeradd, timerlight = list(map(int, l.split()))
                    except ValueError as e:
                        print("ValueError", e)
    except OSError as e:
        print("OSError", e, obj)
        time.sleep_ms(2000)



Sent 93 lines (2968 bytes) to main.py.


In [19]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [20]:
%sendtofile utils.py

import os

def updatehotspots(c="hotspots.txt"):
    while True:
        print("\n\n", c, ":\n")
        with open(c) as fin:
            for x in fin:
                print(x.strip())
        y = input("\ninput line> ")
        ys = y.split()
        if len(ys) != 1 and len(ys) != 4:
            return
        cd = "E"+c
        with open(cd, "w") as fout:
            with open(c) as fin:
                x = None
                for x in fin:
                    xs = x.split()
                    if ys is not None and ys[0] == xs[0]:
                        if len(ys) != 1:
                            fout.write(y)
                            fout.write("\n")
                        ys = None
                    else:
                        fout.write(x)
            if ys is not None and len(ys) != 1:
                if x is not None:
                    fout.write("\n")
                fout.write(y)
                fout.write("\n")
        os.rename(cd, c)
        
print("calling updatehotspots")
updatehotspots()

Sent 34 lines (1031 bytes) to utils.py.


In [21]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-126-g7c2e83324 on 2019-07-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7fd6238c01d0, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [93]:
deviceletter = open("deviceletter.txt", "r").read()
print(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))

b'AAAA'


In [100]:
#print(s.readline())
print(pgled)
pgled.freq(2)
pgled.duty(700)

PWM(23, freq=10, duty=700)


In [94]:
for i in range(20):
    print(s.readline())

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
OSError: [Errno 110] ETIMEDOUT


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [3]:
# for testing UBX is wired in
import machine
uartUBX = machine.UART(1, baudrate=9600, rx=16, tx=17)
print(uartUBX.read())

None


In [4]:
for l in uartUBX:
    print(l)

b'$GNRMC,,V,,,,,,,,,,N*4D\r\n'
b'$GNVTG,,,,,,,,,N*2E\r\n'
b'$GNGGA,,,,,,0,00,99.99,,,,,,*56\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GPGSV,1,1,00*79\r\n'
b'$GLGSV,1,1,00*65\r\n'
b'$GNGLL,,,,,,V,N*7A\r\n'
b'$GNZDA,,,,,00,00*56\r\n'
b'$GNRMC,,V,,,,,,,,,,N*4D\r\n'
b'$GNVTG,,,,,,,,,N*2E\r\n'
b'$GNGGA,,,,,,0,00,99.99,,,,,,*56\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GPGSV,1,1,00*79\r\n'
b'$GLGSV,1,1,00*65\r\n'
b'$GNGLL,,,,,,V,N*7A\r\n'
b'$GNZDA,,,,,00,00*56\r\n'


In [ ]:
%sendtofile main.py

import time, machine, urandom

# delay long enough to interrupt if we need to, 
# before the uart or wlan bricks it for debugging
pled = machine.Pin(2, machine.Pin.OUT)
for i in range(11):
    pled.value(i%2)
    time.sleep_ms(500)  

from ubxminimal import uartUBX, initUBX
initUBX()

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(200)

deviceletter = open("deviceletter.txt", "r").read()
port = {"A":9020, "B":9021, "C":9022}[deviceletter]
udpaddr = ("192.168.43.1", port)  # ip default for android

from UDPblackviewphone import connectBlackview, connectS5
if deviceletter == 'C':
    connectS5(pled)
else:
    connectBlackview(pled)

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(100)


import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.settimeout(0.005)
    
ubxbuffer = bytearray(1000)

# main loop
PCudpaddr = None
reccount = 0
while True:
    n = uartUBX.readinto(ubxbuffer)
    if n is not None:
        try:
            s.sendto(mubxbuffer[:n+2], udpaddr)
            if PCudpaddr is not None:
                s.sendto(mubxbuffer[:n+2], PCudpaddr)
        except OSError as e:
            print("dwrite", e)
        #print(x[:5])
        pled.value(1-pled.value())
    else:
        try:
            mess, PCudpaddr = s.recvfrom(50)
            print(mess, PCudpaddr)
        except OSError as e:
            pass # timeout

